In [3]:
import pandas as pd
import os
import json
import numpy as np

In [1]:
meta = r'C:\Projects\merzlota\to_export\meta.json'
with open (meta, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

holder = []
for name, sensors in metadata.items():
    print(name)
    for sensor in sensors:
        fpath = sensor['path']
        id = sensor['sensor id']
        print(name, id)
        df = pd.read_csv(fpath, sep = '\t')
        df['name'] = str(name)
        df['name'] = df['name'].astype(str)
        df['sensor_id'] = int(id)
        df['date'] = pd.to_datetime(df['date'])
        holder.append(df)
big_df = pd.concat(holder, axis = 0)
big_df

NameError: name 'json' is not defined

In [2]:
big_df['water'].describe()

NameError: name 'big_df' is not defined

In [5]:
big_df.to_pickle(r'C:\Projects\merzlota\predictor\app\data.pck', protocol=1)

In [ ]:
# def make_prediction(name, kind = 'monthly'):
import pandas as pd
import os
import json
import numpy as np

big_df = pd.read_pickle('data.pck')

from datetime  import datetime

def get_next_date(last_date, kind):
    m = last_date.month
    y = last_date.year
    if kind == 'month':

        if m == 12:
            next_y = y + 1
            next_m = 1
        else:
            next_y = y
            next_m = m + 1
        return datetime(next_y, next_m, 1)
    else:
        return datetime(y+1, 1, 1)


period = 20
name = '107.1612'
sensor = 0
kind = 'month'

def get_predict(name, sensor, kind):

    seed = 0
    np.random.seed(seed)
    df = big_df[(big_df['name']==name)&(big_df['sensor_id']==sensor)]
    last_date = df['date'].iloc[-1]
    next_date = get_next_date(last_date, kind)
    dates = [next_date]
    dates_str = [next_date.strftime('%d.%m.%Y')]
    for i in range(period):
        prev_date = dates[-1]
        new_date = get_next_date(prev_date, kind)
        dates.append(new_date)
        dates_str.append(new_date.strftime('%d.%m.%Y'))


    mean_t = df['t'].mean()
    std_t = df['t'].std()
    mean_krens = df['krens'].mean()
    std_krens = df['krens'].std()

    mean_w = df['water'].mean()
    std_w = df['water'].std()

    if kind == 'month':
        noise = 0.1
    else:
        noise = 0.5

    predict_t = np.random.normal(loc = mean_t, scale=std_t + noise, size = period)
    predict_krens = np.random.normal(loc = mean_krens, scale=std_krens + noise, size = period)
    predict_waters = np.random.normal(loc = mean_w, scale=std_w + noise, size = period)

    to_return = {}
    to_return['dates'] = dates_str
    to_return['t'] = list(predict_t)
    to_return['krens'] = list(predict_krens)
    to_return['water'] = list(predict_waters)
    return to_return

get_predict(name, sensor, kind)

In [6]:
import pandas as pd
big_df = pd.read_pickle('data.pck')
big_df

,Unnamed: 0,date,t,krens,water,name,sensor_id
0,0,1971-02-08 11:19:07,-5.569433,-2.956228,13.786,682.3018,0
1,1,1971-02-09 10:49:04,-4.513296,5.885962,4.377,682.3018,0
2,2,1971-02-12 07:34:30,-5.288330,3.777238,0.267,682.3018,0
3,3,1971-03-14 05:14:37,-3.899782,2.095965,14.376,682.3018,0
4,4,1971-03-30 00:55:21,-3.600897,4.206018,11.755,682.3018,0
...,...,...,...,...,...,...,...
502,502,2023-01-26 04:08:53,-5.575927,6.218585,0.134,503.8660,0
503,503,2023-04-28 22:44:38,-4.622386,-6.181698,0.834,503.8660,0
504,504,2023-04-30 12:19:59,-4.921628,-5.762521,7.888,503.8660,0
505,505,2023-07-09 13:40:47,-3.246455,-2.820931,12.745,503.8660,0


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_df(name, sensor):
    df = big_df[(big_df['name']==name)&(big_df['sensor_id']==sensor)]
    fig = make_subplots(3, 1, subplot_titles=['Температура', 'Крен', 'Уровень воды'])

    fig.add_trace(
                go.Scatter(x = df.date, y = df.t, mode = 'lines+markers', 
                hovertemplate = 'Температура: %{y:.2f} <sup>o</sup>С'+ '<br>Дата: %{x}</br>' + '<br>Сенсор #: %{text}</br>',
                text = [sensor]*len(df.date),
                name = f'Sensor #{sensor}',
                showlegend = False,
                ),
                1, 1 )
    fig.add_trace(
                go.Scatter(x = df.date, y = df.krens, mode = 'lines+markers', 
                hovertemplate = 'Температура: %{y:.2f} <sup>o</sup>С'+ '<br>Дата: %{x}</br>' + '<br>Сенсор #: %{text}</br>',
                text = [sensor]*len(df.date),
                name = f'Sensor #{sensor}',
                showlegend = False,
                ),
                2, 1 )   
    fig.add_trace(
                go.Scatter(x = df.date, y = df.water, mode = 'lines+markers', 
                hovertemplate = 'Температура: %{y:.2f} <sup>o</sup>С'+ '<br>Дата: %{x}</br>' + '<br>Сенсор #: %{text}</br>',
                text = [sensor]*len(df.date),
                name = f'Sensor #{sensor}',
                showlegend = False,
                ),
                3, 1 )

    fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    )
    # fig.show()
    return fig.to_html()

# print(get_df('736.7199', 1))

In [12]:
if len(big_df[big_df['name']==1]):
    print('hey')
else:
    print('asd')

asd
